In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [2]:
dtype = torch.FloatTensor

In [3]:
sentences = ["i like dog", "i like cat", "i like animal", "dog cat animal", "apple cat dog like", "dog fish milk like", "dog cat eyes like", "i like apple", "apple i hate", "apple i movie book music like", "cat dog hate", "cat dog like"]

In [4]:
word_sequence = " ".join(sentences).split()
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}

In [5]:
batch_size = 20
embedding_size = 2
voc_size = len(word_dict)

In [6]:
def random_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)
    
    for i in random_index:
        random_inputs.append(np.eye(voc_size)[data[i][0]])
        random_labels.append(data[i][1])
    
    return random_inputs, random_labels

In [7]:
skip_grams = []
for i in range(1, len(word_sequence) - 1):
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]
    
    for w in context:
        skip_grams.append([target, w])

In [8]:
class Word2Vec(nn.Module):
    def __init__(self):
        super(Word2Vec, self).__init__()
        
        self.W = nn.Parameter(-2 * torch.rand(voc_size, embedding_size) + 1).type(dtype) # init the parameter, range of -1 ~ 1
        self.WT = nn.Parameter(-2 * torch.rand(embedding_size, voc_size) + 1).type(dtype)
    
    def forward(self, X):
        hidden_layer = torch.matmul(X, self.W)
        output_layer = torch.matmul(hidden_layer, self.WT)
        return output_layer

In [9]:
model = Word2Vec()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
for epoch in range(5000):
    input_batch, target_batch = random_batch(skip_grams, batch_size)
    
    input_batch = Variable(torch.Tensor(input_batch))
    target_batch = Variable(torch.LongTensor(target_batch))
    
    optimizer.zero_grad()
    output = model(input_batch)
    
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.6f}'.format(loss))
    
    loss.backward()
    optimizer.step()

Epoch:  1000 cost =  1.755965
Epoch:  2000 cost =  2.038094
Epoch:  3000 cost =  1.768003
Epoch:  4000 cost =  1.983617
Epoch:  5000 cost =  1.764826
